In [1]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance

In [ ]:
# INPUT
## d2, d3, d4
timeframe = "3d"
## ema (20 or 25)
ema_size = 25



In [2]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv() # read from local .env file

import os
from binance.client import Client

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']
client = Client(api_key, api_secret, testnet=True)

In [3]:
from binance.spot import Spot

client = Spot()

# Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = client.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

In [4]:
import pandas as pd

# Define function to calculate an arbitrary EMA    
def calc_generic_ema(symbol, timeframe, ema_size):
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
    # Convert into Dataframe
    dataframe = pd.DataFrame(raw_data)
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [5]:
# Prepare df
df = calc_generic_ema("BTCUSDT", timeframe, ema_size)

df['time'] = pd.to_datetime(df['time'], unit='ms')

df=df.round(2)

del df['volume']
del df['close_time']
del df['quote_asset_volume']
del df['number_of_trades']
del df['taker_buy_base_asset_volume']
del df['taker_buy_quote_asset_volume']

df['oversell_day'] = 0
df.loc[df['high'] >= df['ema_25'], 'oversell_day'] = 1


df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
df.loc[df['oversell_day'] == 0, 'streak'] = 0
display(df.tail(30))

,time,open,high,low,close,ema_25,oversell_day,streak
648,2022-12-13,17208.93,18387.95,17080.14,17356.34,18134.90,1,1
649,2022-12-16,17356.96,17531.73,16527.32,16738.21,18027.46,0,0
650,2022-12-19,16739.00,17061.27,16256.30,16824.67,17934.94,0,0
651,2022-12-22,16824.68,16955.14,16559.85,16836.12,17850.41,0,0
652,2022-12-25,16835.73,16972.83,16592.37,16706.36,17762.41,0,0
653,2022-12-28,16706.06,16785.19,16333.00,16607.48,17673.57,0,0
654,2022-12-31,16607.48,16799.23,16470.00,16672.87,17596.59,0,0
655,2023-01-03,16672.78,16991.87,16605.28,16831.85,17537.77,0,0
656,2023-01-06,16831.85,17176.99,16679.00,17127.83,17506.23,0,0
657,2023-01-09,17127.83,18000.00,17104.66,17943.26,17539.85,1,1
